In [1]:
def intersperse(lst, item):
    result = [item] * (len(lst) * 2 + 1)
    result[1::2] = lst
    return result




In [3]:
intersperse([1,2,3,4,5,6], 0)

[0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0]